## Project Final Report

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
set.seed(1234)

## Introduction

A research group in Computer Science at UBC, led by Frank Wood, collected data about how people play video games. They set up a MineCraft server, where the data collected consisted of two datasets: players.csv and sessions.csv. Our report will look at how player characteristics predict their number of played hours on the server. 

Our group was interested in the “kinds” of players that are most likely to contribute a large amount of data, such that they can be targeted in recruiting efforts. To investigate this, we chose the specific research question: **Can gender, experience, and age predict the total number of minutes of playtime from the dataset?** On a broader scale, these insights can help video game companies develop strategies that enhance user experience and engagement across diverse populations.

To answer the question, we must use both datasets provided (players.csv and sessions.csv). This data was collected by the UBC research group by recording play sessions, to collect information on player skill level, past play sessions, and player demographics. The players dataset contains data pertaining to our explanatory variables (gender, experience, age) whilst the sessions dataset contains data pertaining to the response variable (total playtime minutes).

### Datasets

**players.csv**
This dataset contains 196 observations and 7 variables: 
* Experience (character): Player’s level of experience with Minecraft.
* hashedEmail (character): Hashed version of the player’s email, likely for anonymity.
* name (character): Player's name.
* gender(character): Self-reported gender of.
* played_hours (double): Total number of hours a player has spent playing the player.
* age (double): Player's age
* subscribe (logical): Whether or not the player is subscribed to the game-related newsletter

**sessions.csv**
This dataset contains 1535 observations and 5 variables:
* hashedEmail (character): Hashed version of the player’s email, likely for anonymity.
* start_time (character): start date and time of play session.
* end_time (character): end date and time of play session.
* original_start_time (double): start time of a play seesion, in UNIX time.
* original_end_time (double): send of play session, in UNIX time.


**potential issues**
Upon examining the data, there are certain issues which may interfere with our data analysis if left unaddressed: 
* In players.csv, the age column has 2 missing values.
* In sessions.csv, start_time and end_time are in a character format, which should be converted to a proper datetime format and split in order to isolate the total hours played (the difference between end_time and start_time)
* There are outliers (i.e. individuals with extreme playtimes) which can skew the analysis.


## Methods and Results

First, we load in both datasets into our R environment. 

In [ ]:
url_players<-"https://drive.google.com/uc?export=download&id=1M8-duCFfUh3MYjEn0j2WWRXzVMzTG97x"
players <- read_csv(url_players)

url_sessions <-"https://drive.google.com/uc?export=download&id=1DNxV79eDG7KkIWn6hIwgWeAeUN074SCS"
sessions <-read_csv(url_sessions)

We then mutated and wrangled the data by converting the columns to their proper data types in preparation for visualization. In the sessions.csv dataset, we calculated the difference between the start and end time for a particular session to find the number of minutes played. For simplification, we also decided to merge both tidied datasets into one, using “hashedEmail” as the key.

In [ ]:
players_tidy<- players|>
               mutate(gender=as_factor(gender))|>
               mutate(experience= as_factor(experience))|>
               filter(!is.na(played_hours))|>
               filter(!is.na(Age))
               

tidy_sessions <- sessions |>
    separate(col = start_time, into = c("start_date", "start_time"), sep = " ")|>
    separate(col = end_time, into = c("end_date", "end_time"), sep = " ")|>
    mutate(start_time = hm(start_time))|>
    mutate(end_time = hm(end_time))|>
    mutate(played_minutes = end_time-start_time)|>
    mutate(played_minutes = abs(as.numeric(played_minutes)/60))


data_combined<- inner_join(players_tidy, tidy_sessions, by = "hashedEmail")

data_selected<-data_combined|> select(Age, experience, played_minutes, gender)

data_summarized <- data_combined |> 
  summarise(
    n = n(),
    min_age = min(Age, na.rm = TRUE),
    max_age = max(Age, na.rm = TRUE),
    mean_age = mean(Age, na.rm = TRUE),
    median_age = median(Age, na.rm = TRUE),
    min_minutes = min(played_minutes, na.rm = TRUE),
    max_minutes = max(played_minutes, na.rm = TRUE),
    mean_minutes = mean(played_minutes, na.rm = TRUE),
    median_minutes = median(played_minutes, na.rm = TRUE))
data_summarized

data_split <- initial_split(data_selected, prop = 0.70, strata = played_minutes)
data_training <- training(data_split)
data_testing <- testing(data_split)

Now, we summarize our quantitative explanatory variables to understand their basic properties. This provides early insight on the types of individuals that contribute large amounts of data to the server, helping target the research question.

We then proceeded with a few exploratory visualizations of our training dataset to better understand our predictor variables. 


In [ ]:
players_plot1<-data_training|>
               ggplot(aes(x=Age, y=played_minutes,color=experience))+
               geom_point()+
               labs(x="Age", y="Total Time Played (Minutes)", fill="Experience", title="Age vs Total Minutes Played")
players_plot1

players_plot2<-data_training|>
               ggplot(aes(x=Age, y=played_minutes,color=gender))+
               geom_point()+
               labs(x="Age", y="Total Time Played (Minutes)", fill="Gender", title="Age vs Total Minutes Played")
players_plot2

From these visualizations, it is evident that there are outliers in the original dataset – indicated by the data points with exceptionally high play times. In the context of this data, there are several explanations for these outliers. Some sessions may have never been properly closed (ex. Leaving the game open overnight) which can lead to high play times. Related to this, participants who are aware of the study may have played more/left a play session open for longer to generate data intentionally.

To mitigate the effect of outliers on our model’s predicting ability, we omitted play times over 1000 minutes in our final dataset. 

In [ ]:
tidyer_sessions <- sessions |>
    separate(col = start_time, into = c("start_date", "start_time"), sep = " ")|>
    separate(col = end_time, into = c("end_date", "end_time"), sep = " ")|>
    mutate(start_time = hm(start_time))|>
    mutate(end_time = hm(end_time))|>
    mutate(played_minutes = end_time-start_time)|>
    mutate(played_minutes = abs(as.numeric(played_minutes)/60))|>
    filter(played_minutes <= 1000)


data_combined_cleaner<- inner_join(players_tidy, tidyer_sessions, by = "hashedEmail")

data_selected_cleaner<-data_combined_cleaner|> select(Age, experience, played_minutes, gender)

data_summarized_cleaner <- data_combined_cleaner |> 
  summarise(
    n = n(),
    min_age = min(Age, na.rm = TRUE),
    max_age = max(Age, na.rm = TRUE),
    mean_age = mean(Age, na.rm = TRUE),
    median_age = median(Age, na.rm = TRUE),
    min_minutes = min(played_minutes, na.rm = TRUE),
    max_minutes = max(played_minutes, na.rm = TRUE),
    mean_minutes = mean(played_minutes, na.rm = TRUE),
    median_minutes = median(played_minutes, na.rm = TRUE))
data_summarized_cleaner

data_split_cleaner <- initial_split(data_selected_cleaner, prop = 0.70, strata = played_minutes)
data_training_cleaner <- training(data_split_cleaner)
data_testing_cleaner <- testing(data_split_cleaner)

players_plot1_cleaner<-data_training_cleaner|>
               ggplot(aes(x=Age, y=played_minutes,color=experience))+
               geom_point()+
               labs(x="Age", y="Total Time Played (Minutes)", fill="Experience", title="Age vs Total Minutes Played")
players_plot1_cleaner

players_plot2_cleaner<-data_training_cleaner|>
               ggplot(aes(x=Age, y=played_minutes,color=gender))+
               geom_point()+
               labs(x="Age", y="Total Time Played (Minutes)", fill="Gender", title="Age vs Total Minutes Played")
players_plot2_cleaner

From the visualizations, we decided on choosing k-nn regression as our model.
We chose k-nn regression because the relationship between the response variable and the predictor shows a nonlinear pattern. The assumption that was made for the KNN regression is that scaling is needed as the points should be in comparable scales in order to compare and calculate the distance between them. A limitation of this model is its slow performance – it must compute the distance of all the points inside the training set which can take time. In addition, the KNN method is also sensitive to the choice of k (number of neighbours). A value too small can lead to an overly sensitive model (overfitting) whilst a value too large can cause the model to ignore important local patterns (underfitting). 


As mentioned above, we are going to process the data by splitting it into training and testing sets.


There will be one initial split which will divide the data set 70% into the training set and 30% into the testing set. This allows us to train our model on the majority of the data, while reserving a portion for testing to evaluate the model’s performance. Then in the training set, we will perform a 5-fold-cross-validation to choose the correct number of neighbors (k). The split will happen before the modelling and the cross-validation is done on the training data.  The cross-validation happens to ensure that there will be an optimal value of K before testing it, leading to a more stable performance of the k-NN when estimating. 

In [ ]:
knn_recipe <- recipe(played_minutes ~ Age + experience + gender, data = data_training) |>
  step_dummy(all_nominal_predictors()) |> 
  step_zv(all_predictors())|>
  step_normalize(all_predictors())          

knn_model <- nearest_neighbor(mode = "regression",
  neighbors = tune() ) |>
  set_engine("kknn")


knn_workflow <- workflow() |>
  add_model(knn_model) |>
  add_recipe(knn_recipe)

folds <- vfold_cv(data_training, v=5, strata=played_minutes)

knn_grid <- grid_regular(neighbors(range = c(1,25)), levels =10)

knn_results <- tune_grid(
  knn_workflow,
  resamples = folds,
  grid = knn_grid,
  metrics = metric_set(rmse))

best_knn <- knn_results |>
  select_best(metric = "rmse")
final_knn_workflow <- finalize_workflow(knn_workflow, best_knn)

final_knn_fit <- final_knn_workflow |>
  fit(data = data_training)

final_preds <- predict(final_knn_fit, data_testing) |>
  bind_cols(data_testing)

rmse(final_preds, truth = played_minutes, estimate = .pred)

knn_plot<-ggplot(final_preds, aes(x = .pred , y = played_minutes)) +
  geom_point(alpha = 0.6) +
geom_line(data= final_preds, mapping=aes( y=.pred), color = "blue")+
  labs(x = "Actual Played Minutes", y = "Predicted Played Minutes",
       title = "kNN Regression: Actual vs Predicted")
knn_plot